# Restructuring Data into Tidy Form
## 정돈된 형태로 데이터 재구성

- 각 변수는 열을 형성한다
- 각 관측값은 행을 생성한다
- 각 관측 단위별로 별도의 테이블이 구성된다

## 정돈되지 않은 데이터

- 열 이름이 변수 이름이 아니라 값인 경우
- 열 이름에 복수 개의 변수가 저장된 경우
- 변수가 행과 열 모두에 저장된 경우
- 같은 테이블에 복수 형식의 관측 단위가 저장된 경우
- 단일 관측 단위가 복수 테이블에 저장된 경우

### Stack, melt, unstack, pivot

- 정돈된 데이터를 발견하면 pandas 도구를 사용해 데이터를 재구성해야 정돈해야 한다. pandas 에서 정돈을 위해 제공하는 주요 도구는 DataFrame 메서드인 stack, melt, unstack, pivot 이다. 

### rename, rename_axis, reset_index, set_index

- 보다 복잡한 정돈은 텍스트를 완전히 분해해야 한다. str 액세서가 필요다. rename, rename_axis, reset_index, set_index 같은 다른 헬퍼 메서드는 정돈된 데이터를 마지막으로 다듬는 데 도움이 된다.

# Stack
## 8.1 stack 을 이용해 변숫값을 변수 이름으로 정돈

In [4]:
import pandas as pd 
import numpy as np

path = 'C:/Users/HS/Documents/GitHub/Python-Study/Pandas-Cookbook/data'

In [28]:
state_fruit = pd.read_csv(path + '/state_fruit.csv', index_col = 0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


### 방법  
1. 주 이름이 DataFrame 의 인덱스라는 점에 주목.  
이 주들은 수직으로 잘 위치해 있어서 재구성할 필요가 없다. 
문제는 열 이름이다. stack 메서드는 모든 열 이름을 받아 단일 인덱스 라벨로 수직으로 재구성한다. 

In [14]:
# stack 메서드는 모든 열 이름을 받아 단일 인덱스 라벨로 수직으로 재구성
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

2. 이제 MultiIndex 를 가진 Series 가 만들어졌다. 인덱스는 이제 2 레벨이다. 원래의 인덱스는 이전 열 이름을 위한 공간을 만들어주기 위해 왼쪽으로 밀려났다.  
이 명령어 하나를 통해 이제 근본적으로 정돈된 데이터가 형성됐다. 각 변수, 주, 과일, 몸무게는 수직이다. reset_index 메서드를 사용해 결과를 DataFrame 으로 바꿔보자.

In [18]:
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


3. 이제 구조는 올바르지만 열 이름이 무의미하다. 이를 적절한 식별자로 바꾸자.

In [20]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


4. 열 속성을 변경하는 대신 다소 덜 알려져 있지만, rename_axis Series 메서드를 사용해 reset_index 를 사용하기 전에 인덱스 이름을 설정할 수 있다. 

In [25]:
state_fruit.stack().rename_axis(['state', 'fruit']).reset_index(name = 'weight')

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


## 추가 사항 
stack 을 사용할 때의 핵심 중 하나는 인덱스를 변환하지 않고자 하는 것이다. 이 레시피의 데이터셋은 초기에 주를 인덱스로 설정했다. 주를 인덱스로 사용하지 않으면 어떤 일이 발생하는가? 

In [5]:
state_fruit2 = pd.read_csv(path + '/state_fruit2.csv', index_col = 0)
state_fruit2 = state_fruit2.reset_index()
state_fruit2

,Apple,Orange,Banana
State,,,
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [17]:
# 주 이름이 인덱스에 없으므로 이 데이터프레임에 stack 을 사용하면 전체 값을 재구성해 하나의 긴 값을 가진 Series가 된다. 
state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
   Apple           9
   Orange          7
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
dtype: object

In [39]:
# 이 명령어는 이번에는 주 이름을 모함한 모든 열을 재구성했는데, 이는 우리가 원하던 방식이 아니다. 
# 이 데이터를 정확히 재구성하려면 재구성되지 않은 모든 열을 먼저 set_index 메서드를 사용해 인덱스에 배치해야 하고, 
# 그 다음 stack 을 사용해야 한다. 다음 코드는 단계 1 과 비슷한 결과를 나타낸다. 

state_fruit2.set_index('State').stack()

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

# melt
## 8.2 melt 를 사용해 변숫값을 열 이름으로 정돈

- 대부분의 대형 파이썬 라이브러리처럼 pandas 는 동일한 과제를 성취할 수 있는 여러가지 다른 방법이 제공되고 그 차이는 대게 가독성과 성능이다. pandas 에는 melt 라는 DataFrame 메서드가 있어서 앞 레시피에서 설명한 stack 메서드와 비슷한 작동을 하지만, 다소 더 유연한 측면이 있다. 

### 준비 단계 

- 이 레시피에는 melt 메서드를 사용해 변숫값을 가진 간단한 DataFrame 을 열 이름으로 정돈한다. 

### 방법


In [19]:
# 변환이 필요한 열을 파악한다. 
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [49]:
# id_vars 와 value_vars 매개변수에 적절한 열을 전달해 melt 메서드를 사용한다. 
state_fruit2.melt(id_vars = ['State'],  # 열로 유지하고 싶은 이름의 리스트
                  value_vars = ['Apple', 'Orange', 'Banana'])  # value_vars 는 단일 열로 재구성하고 싶은 열들 이름의 리스트

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [20]:
# 이 단계에서 정돈된 데이터가 생성된다. 
# 디폴트로 melt 는 변환 전의 열 이름을 변수로 참고하고 해당 값들은 값으로 참고한다. 
state_fruit2.melt(
                id_vars = ['State'],  # id_vars 는 재구성하지 않고 열로서 유지하고 싶은 이름의 리스트
                value_vars = ['Apple', 'Orange', 'Banana'],  # value_vars 는 단일 열로 재구성하고 싶은 열들의 이름을 가진 리스트
                var_name = 'Fruit',  # var_name : 재구성된 열의 이름 
                value_name = 'Weight')  # value_name : 재구성된 열의 값

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


### 작동원리

melt 메서드는 강력하며 극적으로 DataFrame 을 재구성한다. melt 는 모두 5개의 매개변수를 갖는데 데이터 재구성을 정확히 이해하려면, 이 중 다음 2개를 이해하는 것이 절대적으로 중요하다. 

- id_vars 는 재구성하지 않고 열로서 유지하고 싶은 이름의 리스트
- value_vars 는 단일 열로 재구성하고 싶은 열들의 이름을 가진 리스트
- id_vars (식별변수) 같은 열에 남게 되지만, value_vars에 전달된 각 열에 대해 반복적으로 나타난다.  

melt 의 중요한 측면의 하나는 인덱스의 값을 무시한다는 것이다. 인덱스를 조용히 제거하고 디폴트를 RangeIndex 로 대체한다. 이는 인덱스에 유지하고 싶은 값이 있으면 melt 를 사용하기 전에 인덱스를 리셋해야 한다는 의미이다. 

- 수평 열 이름을 수직 열 이름으로 변환하는 것  : melting, stacking, unpivoting

### 추가사항

melt 메서드의 모든 매개변수는 선택적이므로 모든 값이 단일 여레 배치되고 이전 열 레이블을 또 다른 단일 열에 배치하려면 melt 를 디폴트 값으로 호출해도 된다.

In [22]:
# melt 메서드의 모든 매개변수는 선택적이므로 모든 값이 단일 열에 배치되고 
# 이전 열 레이블을 또 다른 단일 열에 배치하려면 melt 를 디폴트 값으로 호출해도 된다.
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


In [56]:
# melt 가 실질적으로 필요한 변수들이 많아 식별 변수만을 지정하고 싶을 수도 있다. 
# 그런 경우에 melt 를 다음처럼 호출하면 단계 2 와 동일한 결과를 얻을 수 있다. 
# 사실 단일 열을 melt 할 때는 리스트가 필요 없고, 문자열 값만 전달하면 된다. 

state_fruit2.reset_index().melt(id_vars = 'State')

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


# wide_to_long
## 8.3 복수 변수 그룹을 동시에 스태킹

어떤 데이터셋은 열 이름에 동시에 스태킹 돼 개별 열이 돼야 할 복수 그룹의 변수 이름을 사용하고 있다. 영화 데이터셋의 예를 보면 이 경우를 확실히 알게 된다. 우선 모든 배우의 이름과 해당하는 페이스북 좋아요를 갖고 있는 모든 열을 선택해보자.

wide_to_long 함수는 변수 그룹의 이름이 이 레시피에서처럼 모두 동일한 수치로 끝날 때 작동한다.

In [25]:
movie = pd.read_csv(path + '/movie.csv')
movie.head()

actor = movie[['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name', 
              'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']]
actor.head()

# 여기서 변수를 영화 제목(movie_title), 배우이름(actor), 페이스북 좋아요 개수(facebook_likes)로 정의한다면, 
# 두 열의 집합을 개별적으로 스택해야 하는데, 이는 단일 stack 이나 melt 호출로는 불가능하다.


,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


### 준비단계

이 레시피에는 actor Dataframe을 배우 이름과 해당 페이스북 좋아요를 wide_to_long 함수를 사용해 동시에 스태킬하는 방법을 사용해 정돈한다.

### 방법

In [26]:
# 1. 다양한 기능을 가진 wide_to_long 함수를 사용해 데이터를 정된된 형태로 재구성한다. 
# 이 함수를 사용하기 위해 스태킹하려는 열 이름을 바꿔 숫자로 끝나도록 해야 한다. 
# 먼저 사용자 정의 함수를 생성해 열 이름을 변경한다. 

def change_col_name(col_name): 
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx -1:] + col_name[5:fb_idx-1]
    return col_name

In [35]:
col_name = 'actor_facebook_likes_1'
fb_idx = col_name.find('facebook')
print(fb_idx)
col_name = col_name[:5] + col_name[fb_idx -1:] + col_name[5:fb_idx-1]
print(col_name)

6
actor_facebook_likes_1


In [36]:
# 2. 이 함수를 rename 메서드에 전달해 모든 열 이름을 변환한다. 
actor2 = actor.rename(columns = change_col_name)
actor2.head()

,movie_title,actor_1,actor_2,actor_3,actor_facebook_likes_1,actor_facebook_likes_2,actor_facebook_likes_3
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [40]:
# 3. wide_to_long 함수를 사용해 배우와 페이스북 집합을 동시에 스택한다. 

stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2, 
                             stubnames = stubs, 
                             i = ['movie_title'],
                             j = 'actor_num',
                             sep = '_')
actor2_tidy.head(10)

,,actor,actor_facebook_likes
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0
John Carter,1,Daryl Sabara,640.0
Spider-Man 3,1,J.K. Simmons,24000.0
Tangled,1,Brad Garrett,799.0
Avengers: Age of Ultron,1,Chris Hemsworth,26000.0


### 작동원리

wide_to_long 함수는 독특한 방식으로 작동한다. 

- stubnames : 주 매개변수, 문자열의 리스트, 각 문자열은 단일 열 그룹을 나타낸다. 이 문자열로 시작하는 모든 열은 단일 열로 스택된다. 
- 이 레시피에서는 actor 와 actor_face_book_likes 의 두 그룹의 열이 있다. 디폴트로 각 그룹의 열 이름은 숫자로 끝나야 한다. 이 숫자는 이후에 재구성된 데이터의 레이블에 사용된다.

- i : 스택되지 않는 변수를 식별하기 위해 고유한 열인 매개변수 i 를 사용해만 한다. 
- j :  단순히 원래 열 이름 끝에 있는 식별 숫자를 없애 이름을 변경한다. 
- sep : 각 그룹의 열은 아래 밑줄(_)을 사용해 마지막에 붙이는 숫자와 stubname 을 구분한다. 

- 디폴트 prefix 매개변수 : 정규표현식 \d+ 포함, 하나 이상의 숫자를 찾는다.
- suffix


### 추가사항

wide_to_long 함수는 변수 그룹의 이름이 이 레시피에서처럼 모두 동일한 수치로 끝날 때 작동한다.  
변수가 동일한 수치로 끝나지 않거나 숫자로 끝나지 않아도 여전히 열을 동시에 스태킹하기 위해 wide_to_long 을 사용할 수 있다. 

In [41]:
df = pd.read_csv(path + '/stackme.csv', index_col = 0)
df

,Country,a1,b2,Test,d,e
State,,,,,,
TX,US,0.45,0.3,Test1,2,6
MA,US,0.03,1.2,Test2,9,7
ON,CAN,0.70,4.2,Test3,4,2


In [42]:
# 열 a1, b1, d, e 와 같이 스택해보자. 추가적으로 행에 대한 레이블로 a1과 a2를 사용하고자 한다. 
# 이 과제를 해결하려면 열 이름을 바꿔 원하는 레이블로 끝나도록 해야 한다. 

df2 = df.rename(columns = {'a1' : 'group1_a1', 'b2' : 'group1_b2', 
                           'd' : 'group2_a1', 'e' : 'group2_b2'})
df2

,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
State,,,,,,
TX,US,0.45,0.3,Test1,2,6
MA,US,0.03,1.2,Test2,9,7
ON,CAN,0.70,4.2,Test3,4,2


In [45]:
# 이제 suffix 매개변수를 변경할 필요가 있다. 
# 대게 디폴트는 숫자를 찾는 정규식으로 돼 있다. 여기서는 단순히 임의의 개수의 글자를 찾도록 한다. 

pd.wide_to_long(df2.reset_index(), 
               stubnames = ['group1', 'group2'],
               i = ['State', 'Country', 'Test'],
               j = 'Label',
               suffix = '.+',
               sep = '_').reset_index()

,State,Country,Test,Label,group1,group2
0,TX,US,Test1,a1,0.45,2
1,TX,US,Test1,b2,0.30,6
2,MA,US,Test2,a1,0.03,9
3,MA,US,Test2,b2,1.20,7
4,ON,CAN,Test3,a1,0.70,4
5,ON,CAN,Test3,b2,4.20,2
